# 07_Expected_Loss_Stress_Testing
Compute Expected Loss (EL = PD × LGD × EAD) and perform simple stress testing.

In [ ]:
# Common imports for the project
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
sns.set(style='whitegrid')
%matplotlib inline


In [ ]:
import joblib
from src.preprocessing import basic_cleaning
from src.feature_engineering import create_features
import numpy as np

In [ ]:
model_path = '../models/lgbm.joblib'
if not os.path.exists(model_path):
    model_path = '../models/xgb.joblib' if os.path.exists('../models/xgb.joblib') else '../models/logistic.joblib'
print('Model used:', model_path)
pipe = joblib.load(model_path)
df = pd.read_csv('../data/merged_data.csv')
df = basic_cleaning(df)
df = create_features(df)
exclude = ['default','id','index','source']
features = [c for c in df.select_dtypes(include=[np.number]).columns if c not in exclude]
X = df[features].fillna(0)
pd_pred = pipe.predict_proba(X)[:,1]
df['pd_pred'] = pd_pred
display(df[['pd_pred']].describe())

In [ ]:
if 'loan_amnt' in df.columns:
    df['ead'] = df['loan_amnt']
else:
    df['ead'] = 10000
df['lgd'] = 0.45
df['expected_loss'] = df['pd_pred'] * df['lgd'] * df['ead']
display(df[['pd_pred','ead','lgd','expected_loss']].head())

In [ ]:
total_el = df['expected_loss'].sum()
print(f'Total Expected Loss (base case): {total_el:,.2f}')
df['pd_stress'] = np.minimum(df['pd_pred'] * 1.5, 1.0)
df['expected_loss_stress'] = df['pd_stress'] * df['lgd'] * df['ead']
print('Total EL (stress 1.5x PD):', df['expected_loss_stress'].sum())